In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# if torch is not installed:
# !pip install torch

In [18]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [19]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


# 1D helix functions with 4 different parameters
x = r cos(t)

In [20]:
import pandas as pd
import torch
import random

# x = r cos(t)
# 1024 data points, 256 from each helix of a specific r
# each data point contains a x value and a class number idenfiying
#    the helix of a specific r
training_data = torch.zeros(1024,2).numpy()
test_data = torch.zeros(128,2).numpy()
training_labels = torch.zeros(1024).numpy()
test_labels = torch.zeros(128).numpy()
training_data.shape

(1024, 2)

In [21]:
# 4 random r values
rand_r = np.random.rand(4)
rand_r

array([0.55610911, 0.99991772, 0.6191424 , 0.20097477])

In [22]:
class_id = 0
for r in rand_r:
    # generate training data
    t = np.random.rand(256)
    x = r * np.sin(t * np.pi)
    training_labels[class_id*256:(class_id+1)*256] = class_id
    training_data[class_id*256:(class_id+1)*256,0] = t
    training_data[class_id*256:(class_id+1)*256,1] = x

    # generate test data
    t = np.random.rand(32)
    x = r * np.sin(t * np.pi)
    test_labels[class_id*32:(class_id+1)*32] = class_id
    test_data[class_id*32:(class_id+1)*32,0] = t
    test_data[class_id*32:(class_id+1)*32,1] = x
    class_id += 1
training_data

array([[0.23151928, 0.36974835],
       [0.0581334 , 0.10099937],
       [0.93852997, 0.10672608],
       ...,
       [0.08379015, 0.05229464],
       [0.2933401 , 0.160085  ],
       [0.60100216, 0.1909419 ]], dtype=float32)

import matplotlib.pyplot as plt
from cycler import cycler

my_colours = ['steelblue', 'seagreen', 'firebrick']
custom_cycler = cycler(color=my_colours)

fig, ax = plt.subplots()
ax.set_prop_cycle(custom_cycler)
ax.plot()
plt.show()

In [23]:
test_data

array([[0.6823358 , 0.46733925],
       [0.8010265 , 0.32542017],
       [0.9830069 , 0.02967399],
       [0.5634353 , 0.5451025 ],
       [0.96808356, 0.05566679],
       [0.13951053, 0.23600571],
       [0.79191166, 0.33819813],
       [0.50142974, 0.5561035 ],
       [0.9032121 , 0.16650148],
       [0.84283054, 0.26356363],
       [0.5193297 , 0.55508405],
       [0.6712255 , 0.47757307],
       [0.69114596, 0.45881873],
       [0.5828334 , 0.5373855 ],
       [0.01497689, 0.026156  ],
       [0.5023711 , 0.5560937 ],
       [0.7322627 , 0.4145189 ],
       [0.36087844, 0.50383407],
       [0.359702  , 0.5029607 ],
       [0.95538   , 0.07769906],
       [0.7715491 , 0.36572713],
       [0.3706102 , 0.51079416],
       [0.49264273, 0.5559606 ],
       [0.13453175, 0.22810104],
       [0.92351454, 0.13234316],
       [0.25434002, 0.3985533 ],
       [0.43046284, 0.54289204],
       [0.47935647, 0.55494004],
       [0.09226387, 0.15894362],
       [0.01267346, 0.02213554],
       [0.

classes = np.unique(training_data[:,0])

plt.figure()
for c in classes:
    c_train = training_data[training_data[:,0] == c]
    plt.plot(c_train[0],label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

In [24]:
training_data = training_data.reshape((training_data.shape[0], training_data.shape[1], 1))
training_data

array([[[0.23151928],
        [0.36974835]],

       [[0.0581334 ],
        [0.10099937]],

       [[0.93852997],
        [0.10672608]],

       ...,

       [[0.08379015],
        [0.05229464]],

       [[0.2933401 ],
        [0.160085  ]],

       [[0.60100216],
        [0.1909419 ]]], dtype=float32)

In [25]:
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
test_data

array([[[0.6823358 ],
        [0.46733925]],

       [[0.8010265 ],
        [0.32542017]],

       [[0.9830069 ],
        [0.02967399]],

       [[0.5634353 ],
        [0.5451025 ]],

       [[0.96808356],
        [0.05566679]],

       [[0.13951053],
        [0.23600571]],

       [[0.79191166],
        [0.33819813]],

       [[0.50142974],
        [0.5561035 ]],

       [[0.9032121 ],
        [0.16650148]],

       [[0.84283054],
        [0.26356363]],

       [[0.5193297 ],
        [0.55508405]],

       [[0.6712255 ],
        [0.47757307]],

       [[0.69114596],
        [0.45881873]],

       [[0.5828334 ],
        [0.5373855 ]],

       [[0.01497689],
        [0.026156  ]],

       [[0.5023711 ],
        [0.5560937 ]],

       [[0.7322627 ],
        [0.4145189 ]],

       [[0.36087844],
        [0.50383407]],

       [[0.359702  ],
        [0.5029607 ]],

       [[0.95538   ],
        [0.07769906]],

       [[0.7715491 ],
        [0.36572713]],

       [[0.3706102 ],
        [0.5

In [26]:
# shuffle rows in training data and training labels
idx = np.random.permutation(len(training_data))
training_data = training_data[idx]
training_labels = training_labels[idx].astype(int)

In [27]:
# shuffle rows in test data and test labels
idx = np.random.permutation(len(test_data))
test_data = test_data[idx]
test_labels = test_labels[idx].astype(int)

In [ ]:
input_shape = training_data.shape[1:]
n_classes = len(np.unique(training_labels))

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.1,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    training_data,
    training_labels,
    validation_split=0.25,
    epochs=300,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(test_data, test_labels, verbose=1)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 2, 1)]       0           []                               
                                                                                                  
 layer_normalization_32 (LayerN  (None, 2, 1)        2           ['input_5[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_16 (Multi  (None, 2, 1)        7169        ['layer_normalization_32[0][0]', 
 HeadAttention)                                                   'layer_normalization_32[0][0]'] 
                                                                                            

 ambda)                                                           'tf.__operators__.add_37[0][0]']
                                                                                                  
 layer_normalization_39 (LayerN  (None, 2, 1)        2           ['tf.__operators__.add_38[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv1d_38 (Conv1D)             (None, 2, 4)         8           ['layer_normalization_39[0][0]'] 
                                                                                                  
 dropout_43 (Dropout)           (None, 2, 4)         0           ['conv1d_38[0][0]']              
                                                                                                  
 conv1d_39 (Conv1D)             (None, 2, 1)         5           ['dropout_43[0][0]']             
          

12/12 [==============================] - 1s 108ms/step - loss: 1.0074 - sparse_categorical_accuracy: 0.5378 - val_loss: 0.9934 - val_sparse_categorical_accuracy: 0.5742
Epoch 32/300
12/12 [==============================] - 1s 112ms/step - loss: 1.0114 - sparse_categorical_accuracy: 0.5378 - val_loss: 0.9909 - val_sparse_categorical_accuracy: 0.5547
Epoch 33/300
12/12 [==============================] - 1s 119ms/step - loss: 1.0018 - sparse_categorical_accuracy: 0.5312 - val_loss: 0.9771 - val_sparse_categorical_accuracy: 0.5742
Epoch 34/300
12/12 [==============================] - 1s 104ms/step - loss: 0.9941 - sparse_categorical_accuracy: 0.5495 - val_loss: 0.9705 - val_sparse_categorical_accuracy: 0.5664
Epoch 35/300
12/12 [==============================] - 1s 113ms/step - loss: 0.9773 - sparse_categorical_accuracy: 0.5690 - val_loss: 0.9655 - val_sparse_categorical_accuracy: 0.5547
Epoch 36/300
12/12 [==============================] - 1s 120ms/step - loss: 0.9764 - sparse_categorical

12/12 [==============================] - 1s 118ms/step - loss: 0.7634 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7265 - val_sparse_categorical_accuracy: 0.6758
Epoch 77/300
12/12 [==============================] - 2s 192ms/step - loss: 0.7623 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7216 - val_sparse_categorical_accuracy: 0.6641
Epoch 78/300
12/12 [==============================] - 1s 119ms/step - loss: 0.7527 - sparse_categorical_accuracy: 0.6953 - val_loss: 0.7199 - val_sparse_categorical_accuracy: 0.6797
Epoch 79/300
12/12 [==============================] - 1s 112ms/step - loss: 0.7398 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7150 - val_sparse_categorical_accuracy: 0.7031
Epoch 80/300
12/12 [==============================] - 1s 120ms/step - loss: 0.7546 - sparse_categorical_accuracy: 0.6914 - val_loss: 0.7083 - val_sparse_categorical_accuracy: 0.6836
Epoch 81/300
12/12 [==============================] - 1s 120ms/step - loss: 0.7459 - sparse_categorical

Epoch 121/300
12/12 [==============================] - 1s 112ms/step - loss: 0.6556 - sparse_categorical_accuracy: 0.7578 - val_loss: 0.5978 - val_sparse_categorical_accuracy: 0.8359
Epoch 122/300
12/12 [==============================] - 2s 138ms/step - loss: 0.6472 - sparse_categorical_accuracy: 0.7773 - val_loss: 0.5986 - val_sparse_categorical_accuracy: 0.7539
Epoch 123/300
12/12 [==============================] - 1s 122ms/step - loss: 0.6477 - sparse_categorical_accuracy: 0.7669 - val_loss: 0.5962 - val_sparse_categorical_accuracy: 0.7852
Epoch 124/300
12/12 [==============================] - 1s 113ms/step - loss: 0.6494 - sparse_categorical_accuracy: 0.7383 - val_loss: 0.5910 - val_sparse_categorical_accuracy: 0.7930
Epoch 125/300
12/12 [==============================] - 1s 111ms/step - loss: 0.6430 - sparse_categorical_accuracy: 0.7422 - val_loss: 0.5935 - val_sparse_categorical_accuracy: 0.8125
Epoch 126/300
12/12 [==============================] - 1s 120ms/step - loss: 0.6400 -

Epoch 166/300
12/12 [==============================] - 1s 102ms/step - loss: 0.5836 - sparse_categorical_accuracy: 0.8008 - val_loss: 0.5201 - val_sparse_categorical_accuracy: 0.8633
Epoch 167/300
12/12 [==============================] - 1s 121ms/step - loss: 0.5796 - sparse_categorical_accuracy: 0.8034 - val_loss: 0.5198 - val_sparse_categorical_accuracy: 0.8906
Epoch 168/300
12/12 [==============================] - 1s 102ms/step - loss: 0.5795 - sparse_categorical_accuracy: 0.7982 - val_loss: 0.5194 - val_sparse_categorical_accuracy: 0.8828
Epoch 169/300
12/12 [==============================] - 1s 111ms/step - loss: 0.5669 - sparse_categorical_accuracy: 0.8086 - val_loss: 0.5151 - val_sparse_categorical_accuracy: 0.8633
Epoch 170/300
12/12 [==============================] - 1s 111ms/step - loss: 0.5768 - sparse_categorical_accuracy: 0.7865 - val_loss: 0.5200 - val_sparse_categorical_accuracy: 0.8438
Epoch 171/300
12/12 [==============================] - 1s 119ms/step - loss: 0.5879 -

Epoch 211/300
12/12 [==============================] - 1s 109ms/step - loss: 0.5573 - sparse_categorical_accuracy: 0.7865 - val_loss: 0.4787 - val_sparse_categorical_accuracy: 0.8477
Epoch 212/300
12/12 [==============================] - 1s 116ms/step - loss: 0.5283 - sparse_categorical_accuracy: 0.7995 - val_loss: 0.4757 - val_sparse_categorical_accuracy: 0.8516
Epoch 213/300
12/12 [==============================] - 1s 120ms/step - loss: 0.5215 - sparse_categorical_accuracy: 0.8138 - val_loss: 0.4692 - val_sparse_categorical_accuracy: 0.9297
Epoch 214/300
12/12 [==============================] - 1s 110ms/step - loss: 0.5277 - sparse_categorical_accuracy: 0.8047 - val_loss: 0.4580 - val_sparse_categorical_accuracy: 0.9141
Epoch 215/300
12/12 [==============================] - 1s 120ms/step - loss: 0.5124 - sparse_categorical_accuracy: 0.8268 - val_loss: 0.4612 - val_sparse_categorical_accuracy: 0.9297
Epoch 216/300
12/12 [==============================] - 2s 137ms/step - loss: 0.5348 -